In [1]:
import pandas as pd
import os

In [2]:
# load split data
out_dir = "../../data/figshare"
in_dir = "../../data/annotations/split"
splits = ["train", "dev", "test"]

dfs = []
for split in splits:
    df = pd.read_csv(os.path.join(in_dir, split, 'annotations.tsv'), sep = '\t')
    df['split'] = split
    dfs.append(df)
df = pd.concat(dfs)
df.shape

(1368958, 23)

In [3]:
# rename workers
df_workers = df[['_worker_id']].drop_duplicates()
df_workers['anon_id'] = range(df_workers.shape[0])
df = df.merge(df_workers, how = 'inner', on = '_worker_id')
df.shape

(1368958, 24)

In [4]:
# rename some columns
df = df.rename(columns={'clean_diff': 'comment',
                        'diff': 'raw_comment',
                        'rev_timestamp': 'timestamp',
                        'rev_id': 'comment_id',
                        'anon_id': 'worker_id',
                       })

In [5]:
# get set of labeled comments
df_comments = df.drop_duplicates(subset = ['comment_id'])
order = ['comment_id', 'comment', 'raw_comment', 'timestamp', 'page_id', 'page_title', 'user_id', 'user_text', 'ns', 'sample', 'split']
df_comments = df_comments[order]
df_comments = df_comments.sort_values('comment_id')
df_comments.shape

(116179, 11)

In [6]:
# get set of human labels
df_labels = df[['comment_id', 'worker_id', 'attack', 'aggression']]
df_labels = df_labels.sort_values('comment_id')
df_labels.shape

(1368958, 4)

In [7]:
# save dfs
df_comments.to_csv(os.path.join( "../../data/figshare", 'crowd_annotated_comments.tsv'), sep = '\t', index = False)
df_labels.to_csv(os.path.join( "../../data/figshare", 'crowd_annotations.tsv'), sep = '\t', index = False)